In [1]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import sklearn.ensemble
from sklearn.metrics import accuracy_score, roc_auc_score, balanced_accuracy_score, precision_score, matthews_corrcoef, confusion_matrix, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import statistics
import numpy as np
import subprocess
from pandarallel import pandarallel

In [2]:
df = pd.read_csv("data/4.2_hg38_paired_introns.tsv", sep="\t", usecols = ["sequence_maxentscan"])
df

,sequence_maxentscan
0,ccaGTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGC...
1,gagGTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTA...
2,gagCACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAG...
3,cttGTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAG...
4,tagGCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGG...
...,...
499138,aatGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAGGC...
499139,cagGTAATGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATT...
499140,gggATTATGTTTTCCTTGATGTTAAGTGAATTAGCCAAACATAGAC...
499141,cgcCCTTGGTTTTCCTTACACCTTAGCCTTTGGCTCCTTTGACCAC...


In [3]:
df["5_prime_site_maxent_seq"] = df["sequence_maxentscan"].apply(lambda x: x[:9])
df

,sequence_maxentscan,5_prime_site_maxent_seq
0,ccaGTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGC...,ccaGTAAGT
1,gagGTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTA...,gagGTGAGA
2,gagCACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAG...,gagCACTGG
3,cttGTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAG...,cttGTGAGT
4,tagGCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGG...,tagGCAAGC
...,...,...
499138,aatGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAGGC...,aatGTAAGA
499139,cagGTAATGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATT...,cagGTAATG
499140,gggATTATGTTTTCCTTGATGTTAAGTGAATTAGCCAAACATAGAC...,gggATTATG
499141,cgcCCTTGGTTTTCCTTACACCTTAGCCTTTGGCTCCTTTGACCAC...,cgcCCTTGG


In [4]:
df["3_prime_site_maxent_seq"] = df["sequence_maxentscan"].apply(lambda x: x[-23:])
df

,sequence_maxentscan,5_prime_site_maxent_seq,3_prime_site_maxent_seq
0,ccaGTAAGTAGTGCTTGTGCTCATCTCCTTGGCTGTGATACGTGGC...,ccaGTAAGT,AGGCTCCTGTCTCCCCCCAGgtg
1,gagGTGAGAGGAGAGTAGACAGTGAGTGGGAGTGGCGTCGCCCCTA...,gagGTGAGA,TAGCCTCTGTTCCCACGAAGgca
2,gagCACTGGAGTGGAGTTTTCCTGTGGAGAGGAGCCATGCCTAGAG...,gagCACTGG,AACCAGGCATAGGGGAAAGAttg
3,cttGTGAGTGTCCCCAGTGTTGCAGAGGTGAGAGGAGAGTAGACAG...,cttGTGAGT,GTCATCCCCTTCACTCCCAGctc
4,tagGCAAGCCTGGCTGCCTCCAGCTGGGTCGACAGACAGGGGCTGG...,tagGCAAGC,TAGCCTCTGTTCCCACGAAGgca
...,...,...,...
499138,aatGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATTGAGGC...,aatGTAAGA,CCATATTCATTTTTTTCTGCcaa
499139,cagGTAATGTAAGAAGGAGTAAAATTATTTGCTTTCAGGTATTATT...,cagGTAATG,CTTTTTTTTTTTTTTTTAAGatg
499140,gggATTATGTTTTCCTTGATGTTAAGTGAATTAGCCAAACATAGAC...,gggATTATG,TGAAAACTTCTGTGTTTCAGaaa
499141,cgcCCTTGGTTTTCCTTACACCTTAGCCTTTGGCTCCTTTGACCAC...,cgcCCTTGG,GTCTACTTTCCAAATTACAGtct


In [5]:
df=df.drop(columns=["sequence_maxentscan"], axis=1)
df

,5_prime_site_maxent_seq,3_prime_site_maxent_seq
0,ccaGTAAGT,AGGCTCCTGTCTCCCCCCAGgtg
1,gagGTGAGA,TAGCCTCTGTTCCCACGAAGgca
2,gagCACTGG,AACCAGGCATAGGGGAAAGAttg
3,cttGTGAGT,GTCATCCCCTTCACTCCCAGctc
4,tagGCAAGC,TAGCCTCTGTTCCCACGAAGgca
...,...,...
499138,aatGTAAGA,CCATATTCATTTTTTTCTGCcaa
499139,cagGTAATG,CTTTTTTTTTTTTTTTTAAGatg
499140,gggATTATG,TGAAAACTTCTGTGTTTCAGaaa
499141,cgcCCTTGG,GTCTACTTTCCAAATTACAGtct


In [6]:
pandarallel.initialize()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [7]:
def maxentscan_5(row):
    """
    We call the tabix command, which look at the bed file to see if the row (which are entries in the intron that we extracted) overlaps with the repeat
    regions detailed in the repeat_features.bed.gz, apart from the splice-site ({row.end-2}), we also look at the small-region that proceeds (2 nt in the exon; {row.end+2}) the splice-site.
    """
    # score = !perl MaxEntScan/score5.pl {row.sequence[:9]} | cut -f2
    # # score = score[-1:1]
    # return score
    target_sequence = row
    if len(target_sequence) < 9:
	    return 0
    result = subprocess.run(f"perl MaxEntScan/score5.pl {target_sequence}", shell=True, stdout=subprocess.PIPE, text=True)
    try:
    	score = result.stdout.strip().split("\t")[1]
    except:
        print(result)
        return result
    return score

In [8]:
%%time
df_test = df.head(5000)
df_test["5_prime_site_maxent_seq"].parallel_apply(maxentscan_5)
del df_test

CPU times: user 32.4 ms, sys: 219 ms, total: 252 ms
Wall time: 1min


In [9]:
%%time
df["5_MaxEntScan"] = df["5_prime_site_maxent_seq"].parallel_apply(maxentscan_5)

CPU times: user 274 ms, sys: 276 ms, total: 549 ms
Wall time: 1h 59min 21s


In [10]:
df.loc[df["5_prime_site_maxent_seq"].str.contains("N"), "5_MaxEntScan"] = 0
df[df["5_prime_site_maxent_seq"].str.contains("N")]

,5_prime_site_maxent_seq,3_prime_site_maxent_seq,5_MaxEntScan


# 3-prime end

In [11]:
def maxentscan_3(row):
    """
    We call the tabix command, which look at the bed file to see if the row (which are entries in the intron that we extracted) overlaps with the repeat
    regions detailed in the repeat_features.bed.gz, apart from the splice-site ({row.end-2}), we also look at the small-region that proceeds (2 nt in the exon; {row.end+2}) the splice-site.
    """
    target_sequence = row
    if len(target_sequence) < 23:
	    return 0
    result = subprocess.run(f"perl MaxEntScan/score3.pl {target_sequence}", shell=True, stdout=subprocess.PIPE, text=True)
    try:
    	score = result.stdout.strip().split("\t")[1]
    except:
        return 0
    return score

In [12]:
df[df["3_prime_site_maxent_seq"].str.len() < 23]

,5_prime_site_maxent_seq,3_prime_site_maxent_seq,5_MaxEntScan
31,gaaCTGAct,gaaCTGActa,-7.85
9968,cggGccc,cggGccc,0
10255,gctAAGCTA,gctAAGCTAATGCTaat,-30.86
10257,cggCAGTGA,cggCAGTGATGGCAtgt,-13.67
11250,cacCTGAGg,cacCTGAGgtc,-2.49
...,...,...,...
392156,gggCCTGAC,gggCCTGACTCCCTcac,-26.47
395121,tcgCCcaa,tcgCCcaa,0
396609,cgaCTtcc,cgaCTtcc,0
397719,ctaATttt,ctaATttt,0


In [13]:
%%time
df["3_MaxEntScan"] = df["3_prime_site_maxent_seq"].parallel_apply(maxentscan_3)

CPU times: user 289 ms, sys: 363 ms, total: 652 ms
Wall time: 2h 1min 29s


In [14]:
df.loc[df["3_prime_site_maxent_seq"].str.contains("N"), "3_MaxEntScan"] = 0
df[df["3_prime_site_maxent_seq"].str.contains("N")]

,5_prime_site_maxent_seq,3_prime_site_maxent_seq,5_MaxEntScan,3_MaxEntScan


Output to .tsv

In [15]:
df.to_csv("data/4.3_maxentscan", sep="\t", index=False)